<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**MODELOS DE PROGRAMACION: MODELO ADIABATICO** -
Prof: Carmen Pellicer Lostao

# Constrained Scheduling

## Intro

This notebook solves a **binary constraint satisfaction problem (CSP)**. CSPs require that all a problem’s variables be assigned values that result in the satisfying of all constraints. Here, the constraints are a company’s policy for scheduling meetings:

- Constraint 1: During business hours, all meetings must be attended in person at the office.

- Constraint 2: During business hours, participation in meetings is mandatory.

- Constraint 3: Outside business hours, meetings must be teleconferenced.

- Constraint 4: Outside business hours, meetings must not exceed 30 minutes.

Solving such a CSP means **finding arrangements of meetings that meet all the constraints**.

The purpose of this example is to help a new user to formulate a constraint satisfaction problem using Ocean tools and solve it on a D-Wave quantum computer.

This example follows the [Workflow steps: Formulation and Sampling](https://docs.ocean.dwavesys.com/en/stable/overview/solving_problems.html#solving-problems) that describes the problem-solving workflow as consisting of two main steps:

1.  **Formulate the problem** as an objective function in a supported form of quadratic model (QM) and
2.   **Solve your QM** with a D-Wave solver.

In CSP problems we represent the **problem’s constraints as penalties** (small Binary Quadratic Modelss that have higher values for variable assignments that violate constraints) and **creates an objective function by summing all four penalty models**. Solvers that seek low-energy states are thus less likely to return meeting arrangements that violate constraints.

## Formulate the problem

D-Wave quantum computers solve binary quadratic models, so the first step is to express the problem with binary variables

### The Variables

In this scheduling example we use the following valued binary variables:

Problem Variables

|Variable | Represents | Value: 1 | Value: 0 |
| :---: | :---: | :---: | :---: |
|t |Time of day|Business hours|Non-business hours|
|v|Venue|Office|Teleconference|
|l|Length|Short (< 30 min)|Long|
|p|Participation|Mandatory|Optional|



<u>**NOTE**</u>: A slightly more complex problem might require variables with multiple values; for example, l could have values {30, 60, 120} representing the duration in minutes of meetings of several lengths. For such problems a discrete quadratic model (DQM) could be a better choice.

In general, problems with constraints are more simply solved using a constrained quadratic model (CQM).

For large numbers of variables and constraints, such problems can be hard. This example has four binary variables, so only $2^4=16$ possible meeting arrangements. As shown in the table below, it is a simple matter to work out all the combinations by hand to find solutions that meet all the constraints.

All Possible Meeting Options.

|Time of Day|Venue|Duration|Participation|Valid?|
| :---: | :---: | :---: | :---: | :---: |
|Business hours|Office|Short|Mandatory|Yes|
|Business hours|Office|Short|Optional|No (violates 2)|
|Business hours|Office|Long|Mandatory|Yes|
|Business hours|Office|Long|Optional|No (violates 2)|
|Business hours|Teleconference|Short|Mandatory|No (violates 1)|
|Business hours|Teleconference|Short|Optional|No (violates 1, 2)|
|Business hours|Teleconference|Long|Mandatory|No (violates 1)|
|Business hours|Teleconference|Long|Optional|No (violates 1, 2)|
|Non-business hours|Office|Short|Mandatory|No (violates 3)|
|Non-business hours|Office|Short|Optional|No (violates 3)|
|Non-business hours|Office|Long|Mandatory|No (violates 3, 4)|
|Non-business hours|Office|Long|Optional|No (violates 3, 4)|
|Non-business hours|Teleconference|Short|Mandatory|Yes|
|Non-business hours|Teleconference|Short|Optional|Yes|
|Non-business hours|Teleconference|Long|Mandatory|No (violates 4)|
|Non-business hours|Teleconference|Long|Optional|No (violates 4)|

### Represent Constraints as Penalties

You can represent constraints as BQMs using Penalty Models in many different ways.

- Constraint 1: During business hours, all meetings must be attended in person at the office.

This constraint requires that if $t=1$ (time of day is within business hours) then $v=1$ (venue is the office). A simple penalty function,
, is shown in the truth table below:

|$t$|$v$| $t-tv$|
| :---: | :---: | :---: |
|0|0|0|
|0|1|0|
|1|0|1|
|1|1|0|

<u>**NOTE**</u>: One way to derive such a penalty function is to start with the simple case of a Boolean operator: the AND constraint, $ab$, penalizes variable values $a=b=1$. To penalize $a=1,b=0$ , you need the penalty function $a¬b$
. For binary valued variables, you can substitute $~b=(1-b)$ into the penalty and get the penalty $a(1-b)$

- Constraint 2: During business hours, participation in meetings is mandatory.

This constraint requires that if $t=1$
 (time of day is within business hours) then $p=1$ (participation is mandatory). A penalty function is $t-tp$, analogous to constraint 1.

- Constraint 3: Outside business hours, meetings must be teleconferenced.

This constraint requires that if $t=0$ (time of day is outside business hours) then $v=0$
 (venue is teleconference, not the office). A penalty function is %v-tv$, a reversal of constraint 1.

- Constraint 4: Outside business hours, meetings must not exceed 30 minutes.

This constraint requires that if $t=0$ (time of day is outside business hours) then $l=1$ (meeting length is short). A simple penalty function is $1+tl-t-l$
, as shown in the truth table below:

|$t$|$v$| $1+tl-t-l$    |
| :---: | :---: | :---: |
|0|0|1                  |
|0|1|0                  |
|1|0|0                  |
|1|1|0                  |

Penalty function $1+tl-t-l$ is
 sets a penalty of 1 for the case $t=l=0$
, representing a lengthy meeting outside business hours, which violates constraint 4. When incorporated in an objective function, solutions that violate constraint 4 do not yield minimal values

<u>**NOTE**</u>: One way to derive such a penalty function is to start with the simple case of a Boolean operator: the AND constraint, $ab$, penalizes variable values $a=b=1$. To penalize $a=0,b=0$ , you need the penalty function $~a~b$
. For binary valued variables, you can substitute $¬a=(1-a)$ and $¬b=(1-b)$into the
penalty and get the penalty $(1-a)(1-b)=1-a-b+ab$

### Create a BQM

The total penalty for all four constraints is

$t-tv+t-tp+v-tv+1+tl-t-l = -2tv-tp+tl+t+v-l+1$

Ocean’s dimod enables the creation of BQMs. Below, the first list of terms are the linear terms and the second are the quadratic terms; the offset is set to 1; and the variable type is set to use
–valued binary variables

In [ ]:
from dimod import BinaryQuadraticModel
bqm = BinaryQuadraticModel({'t': 1, 'v': 1, 'l': -1},       #linear
                           {'tv': -2, 'tl': 1, 'tp': -1},   #quadratic
                           1,                               #offset
                           'BINARY')                        #QUBO

## Solve the problem in a QPU

For small numbers of variables, even your computer’s CPU can solve CSPs quickly. Here you solve both classically on your CPU and on the quantum computer.

### Solving classically on a CPU

Before using a D-Wave quantum computer, it can sometimes be helpful to test code locally. Here, select one of Ocean software’s [classical samplers](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampler_composites/samplers.html) or [dimod sampler utilities](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampler_composites/index.html) to solve classically on a CPU.

Ocean’s `dimod` [ExactSolver()](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampler_composites/samplers.html#exact-solver) provides a sampler that simply returns the BQM’s value (energy) for every possible assignment of variable values.

Ocean's `dwave-samplers` [SimulatedAnnealing()](https://docs.ocean.dwavesys.com/en/stable/docs_samplers/index.html#simulated-annealing) provides a sampler that simulates a QPU

Valid solutions—assignments of variables that do not violate constraints—have the lowest value of the BQM (values of zero in the energy field below):


In [ ]:
from dimod.reference.samplers import ExactSolver
sampler = ExactSolver()
sampleset_exact = sampler.sample(bqm)
print(sampleset_exact)
print(print(sampleset_exact.lowest(atol=.5)))

In [ ]:
from dwave.samplers import SimulatedAnnealingSampler
sampler = SimulatedAnnealingSampler()
sampleset_sim = sampler.sample(bqm)
print(sampleset_sim)
print(print(sampleset_sim.lowest(atol=.5)))

The code below prints all those solutions (assignments of variables) for which the BQM has its minimum value.

In [ ]:
for sample, energy in sampleset_exact.data(['sample', 'energy']):
    if energy==0:
        time = 'business hours' if sample['t'] else 'evenings'
        venue = 'office' if sample['v'] else 'home'
        length = 'short' if sample['l'] else 'long'
        participation = 'mandatory' if sample['p'] else 'optional'
        print("During {} at {}, you can schedule a {} meeting that is {}".format(time, venue, length, participation))

### Solving on a D-Wave Quantum Computer (QPU)

Now solve on a D-Wave system using sampler [DWaveSampler()](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#dwavesampler) from Ocean software’s dwave-system. Also use its [EmbeddingComposite()](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/composites.html#embeddingcomposite) composite to map your unstructured problem (variables such as t etc.) to the sampler’s graph structure (the QPU’s numerically indexed qubits) in a process known as minor-embedding. The next code sets up a D-Wave quantum computer as the sampler.

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())

Because the sampled solution is probabilistic, returned solutions may differ between runs. Typically, when submitting a problem to a quantum computer, you ask for many samples, not just one. This way, you see multiple “best” answers and reduce the probability of settling on a suboptimal answer. Below, ask for 5000 samples.

In [ ]:
sampleset = sampler.sample(bqm, num_reads=5000, label='SDK Examples - Scheduling')

The code below prints all those solutions (assignments of variables) for which the BQM has its minimum value and the number of times it was found.

In [ ]:
print(sampleset.lowest(atol=.5))

In [ ]:
for sample, energy in sampleset.data(['sample', 'energy']):
    if energy==0:
        time = 'business hours' if sample['t'] else 'evenings'
        venue = 'office' if sample['v'] else 'home'
        length = 'short' if sample['l'] else 'long'
        participation = 'mandatory' if sample['p'] else 'optional'
        print("During {} at {}, you can schedule a {} meeting that is {}".format(time, venue, length, participation))

### Solving with DWave Hybrid Solvers

For big problems with many variables it is useful to use Hybrid solvers. The use of this solvers is quite straight forward and similiar with the SAPI interface.

Let's solve this problem with the [LeapHybridSampler](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#leaphybridsampler), a class for using Leap’s cloud-based hybrid BQM solvers. Leap’s quantum-classical hybrid BQM solvers are intended to solve arbitrary application problems formulated as binary quadratic models (BQM).

In [ ]:
from dwave.system import LeapHybridSampler

# Find a good solution
sampler = LeapHybridSampler()       
sampleset = sampler.sample(bqm) 

In [ ]:
sampler.properties

In [ ]:
print(sampleset.lowest(atol=.5))

In [ ]:
for sample, energy in sampleset.data(['sample', 'energy']):
    if energy==0:
        time = 'business hours' if sample['t'] else 'evenings'
        venue = 'office' if sample['v'] else 'home'
        length = 'short' if sample['l'] else 'long'
        participation = 'mandatory' if sample['p'] else 'optional'
        print("During {} at {}, you can schedule a {} meeting that is {}".format(time, venue, length, participation))